last implimentation

In [1]:
import hopsworks
# Connect to Hopsworks Feature Store
project = hopsworks.login()
fs = project.get_feature_store()
# Load from Hopsworks feature groups
users_fg = fs.get_feature_group(name="users", version=1)
events_fg = fs.get_feature_group(name="events", version=1)
interactions_fg = fs.get_feature_group(name="interactions", version=1)

#read data from hopsworks
users_df = users_fg.read()
events_df = events_fg.read()
interactions_df =interactions_fg.read()

2025-05-26 22:08:21,387 INFO: Initializing external client
2025-05-26 22:08:21,389 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-26 22:08:22,944 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1220788
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.14s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.23s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (7.10s) 


In [2]:
users_df.head()

,user_id,user_lat,user_lon,user_city,indoor_outdoor_preference,age,user_interests,signup_date,social_connectedness
0,RC268C,40.962648,-73.613922,New York,outdoor,53,sports fashion fitness,2024-09-15 17:37:26.277721+00:00,13
1,OH989Z,35.366580,139.371354,Tokyo,outdoor,21,art literature travel fitness,2024-09-13 07:24:04.467340+00:00,13
2,RA922W,35.851979,139.316280,Tokyo,outdoor,45,fitness tech food art,2025-02-14 21:46:33.410035+00:00,18
3,EE813A,40.518719,-74.162980,New York,any,55,sports fitness art,2024-02-20 06:27:12.752684+00:00,20
4,FX457B,43.796518,-79.489105,Toronto,any,42,music food,2023-10-12 08:01:57.659010+00:00,20


In [ ]:
events_df.head()

In [4]:
interactions_df.head()

,interaction_id,user_id,event_id,interaction_type,interaction_time,interaction_distance_to_event,interaction_label
0,MS115P,JQ744Q,WC393C,yes,2025-06-14 12:11:15.415927+00:00,42.471559,1
1,FA113Q,KD996K,BL473T,no,2025-06-14 19:38:16.109952+00:00,70.228559,0
2,IR265L,NT251J,LG400F,invited & maybe,2025-08-03 05:49:34.516328+00:00,8.747134,1
3,MY826W,JT337Z,CW093D,no,2025-06-07 16:19:05.147064+00:00,90.449921,0
4,OZ813P,JX049Z,UO577O,maybe,2025-06-21 11:26:39.643373+00:00,36.451322,1


In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from surprise import Dataset, Reader, SVD
from scipy.sparse import hstack
from catboost import CatBoostClassifier

# --- Metrics ---
K_LIST = [1, 5, 10, 50, 100]

def compute_ranking_metrics(y_true, y_score, k=10):
    sorted_indices = np.argsort(y_score)[::-1]
    top_k = np.array(y_true)[sorted_indices][:k]
    precision = np.mean(top_k)
    recall = np.sum(top_k) / np.sum(y_true) if np.sum(y_true) > 0 else 0
    dcg = np.sum(top_k / np.log2(np.arange(2, len(top_k) + 2)))
    ideal_k = min(int(np.sum(y_true)), k)
    idcg = np.sum([1 / np.log2(i + 2) for i in range(ideal_k)])
    ndcg = dcg / idcg if idcg > 0 else 0
    return precision, recall, ndcg

def mean_reciprocal_rank(y_true, y_score, k=100):
    sorted_indices = np.argsort(y_score)[::-1][:k]
    top_k = np.array(y_true)[sorted_indices]
    ranks = np.where(top_k == 1)[0]
    if len(ranks) == 0:
        return 0.0
    return 1.0 / (ranks[0] + 1)

def compute_all_metrics(y_true, y_score, k_list=K_LIST):
    metrics = {}
    for k in k_list:
        p, r, n = compute_ranking_metrics(y_true, y_score, k)
        metrics[f"Precision@{k}"] = p
        metrics[f"Recall@{k}"] = r
        metrics[f"NDCG@{k}"] = n
        metrics[f"MRR@{k}"] = mean_reciprocal_rank(y_true, y_score, k)
    return metrics

# --- Model Training ---
def train_content_model(train_df, val_df):
    train_clean = train_df.copy()
    val_clean = val_df.copy()
    for df in [train_clean, val_clean]:
        df["title"] = df["title"].fillna("")
        df["user_interests"] = df["user_interests"].fillna("")
    tfidf_title = TfidfVectorizer(max_features=100)
    tfidf_interests = TfidfVectorizer(max_features=100)
    tfidf_title_mat_train = tfidf_title.fit_transform(train_clean["title"])
    tfidf_interests_mat_train = tfidf_interests.fit_transform(train_clean["user_interests"])
    tfidf_title_mat_val = tfidf_title.transform(val_clean["title"])
    tfidf_interests_mat_val = tfidf_interests.transform(val_clean["user_interests"])
    numeric_cols = ["interaction_distance_to_event", "temperature", "age", "attendance_rate"]
    for df in [train_clean, val_clean]:
        df[numeric_cols] = df[numeric_cols].fillna(0)
    scaler = StandardScaler().fit(train_clean[numeric_cols])
    X_numeric_train = scaler.transform(train_clean[numeric_cols])
    X_numeric_val = scaler.transform(val_clean[numeric_cols])
    categorical_cols = ['weather_condition', 'event_indoor_capability', 'indoor_outdoor_preference']
    for df in [train_clean, val_clean]:
        for col in categorical_cols:
            df[col] = df[col].fillna('unknown')
    encoder = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
    X_cat_train = encoder.fit_transform(train_clean[categorical_cols])
    X_cat_val = encoder.transform(val_clean[categorical_cols])
    X_train = hstack([tfidf_title_mat_train, tfidf_interests_mat_train, X_numeric_train, X_cat_train]).toarray()
    X_val = hstack([tfidf_title_mat_val, tfidf_interests_mat_val, X_numeric_val, X_cat_val]).toarray()
    y_train = train_clean["interaction_label"].astype(int)
    y_val = val_clean["interaction_label"].astype(int)
    model = CatBoostClassifier(iterations=200, depth=6, learning_rate=0.1, loss_function='Logloss', verbose=False)
    model.fit(X_train, y_train)
    val_scores = model.predict_proba(X_val)[:, 1]
    return model, compute_all_metrics(y_val, val_scores), tfidf_title, tfidf_interests, scaler, encoder

def train_svd(train_df, val_df):
    reader = Reader(rating_scale=(0, 1))
    train_svd = train_df[["user_id", "event_id", "interaction_label"]].copy()
    train_svd["user_id"] = train_svd["user_id"].astype(str)
    train_svd["event_id"] = train_svd["event_id"].astype(str)
    train_svd["interaction_label"] = train_svd["interaction_label"].astype(float)
    data = Dataset.load_from_df(train_svd, reader)
    trainset = data.build_full_trainset()
    svd = SVD(n_epochs=50).fit(trainset)
    val_copy = val_df.copy()
    val_copy["svd_score"] = val_copy.apply(
        lambda row: svd.predict(str(row["user_id"]), str(row["event_id"])).est, axis=1
    )
    y_val = val_copy["interaction_label"].astype(int)
    val_scores = val_copy["svd_score"]
    return svd, compute_all_metrics(y_val, val_scores)

def train_hybrid_model(train_df, val_df):
    reader = Reader(rating_scale=(0, 1))
    train_svd_data = Dataset.load_from_df(train_df[["user_id", "event_id", "interaction_label"]].astype(str), reader)
    trainset = train_svd_data.build_full_trainset()
    svd_model = SVD(n_epochs=20).fit(trainset)
    train_df = train_df.copy()
    val_df = val_df.copy()
    train_df["svd_score"] = train_df.apply(
        lambda row: svd_model.predict(str(row["user_id"]), str(row["event_id"])).est, axis=1
    )
    val_df["svd_score"] = val_df.apply(
        lambda row: svd_model.predict(str(row["user_id"]), str(row["event_id"])).est, axis=1
    )
    tfidf_title = TfidfVectorizer(max_features=50)
    tfidf_interests = TfidfVectorizer(max_features=50)
    tfidf_title.fit(train_df["title"].fillna(""))
    tfidf_interests.fit(train_df["user_interests"].fillna(""))
    X_train_text = hstack([
        tfidf_title.transform(train_df["title"].fillna("")),
        tfidf_interests.transform(train_df["user_interests"].fillna(""))
    ])
    X_val_text = hstack([
        tfidf_title.transform(val_df["title"].fillna("")),
        tfidf_interests.transform(val_df["user_interests"].fillna(""))
    ])
    numeric_cols = ["interaction_distance_to_event", "temperature", "age", "attendance_rate", "svd_score"]
    for df in [train_df, val_df]:
        df[numeric_cols] = df[numeric_cols].fillna(0)
    scaler = StandardScaler().fit(train_df[numeric_cols])
    X_train_numeric = scaler.transform(train_df[numeric_cols])
    X_val_numeric = scaler.transform(val_df[numeric_cols])
    categorical_cols = ['weather_condition', 'event_indoor_capability', 'indoor_outdoor_preference']
    for df in [train_df, val_df]:
        for col in categorical_cols:
            df[col] = df[col].fillna('unknown')
    encoder = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
    X_cat_train = encoder.fit_transform(train_df[categorical_cols])
    X_cat_val = encoder.transform(val_df[categorical_cols])
    X_train = hstack([X_train_text, X_train_numeric, X_cat_train]).toarray()
    X_val = hstack([X_val_text, X_val_numeric, X_cat_val]).toarray()
    y_train = train_df["interaction_label"].astype(int)
    y_val = val_df["interaction_label"].astype(int)
    model = CatBoostClassifier(iterations=200, depth=6, learning_rate=0.1, loss_function='Logloss', verbose=False)
    model.fit(X_train, y_train)
    val_scores = model.predict_proba(X_val)[:, 1]
    return model, compute_all_metrics(y_val, val_scores), tfidf_title, tfidf_interests, scaler, encoder, svd_model

# --- Test Set Evaluation ---
def evaluate_content_or_hybrid_on_test(test_df, model, tfidf_title, tfidf_interests, scaler, encoder, model_type="content", svd_model=None, k_list=K_LIST):
    df = test_df.copy()
    df["title"] = df["title"].fillna("")
    df["user_interests"] = df["user_interests"].fillna("")
    if model_type == "hybrid":
        df["svd_score"] = df.apply(
            lambda row: svd_model.predict(str(row["user_id"]), str(row["event_id"])).est, axis=1
        )
        numeric_cols = ["interaction_distance_to_event", "temperature", "age", "attendance_rate", "svd_score"]
    else:
        numeric_cols = ["interaction_distance_to_event", "temperature", "age", "attendance_rate"]
    df[numeric_cols] = df[numeric_cols].fillna(0)
    tfidf_title_mat = tfidf_title.transform(df["title"])
    tfidf_interests_mat = tfidf_interests.transform(df["user_interests"])
    X_numeric = scaler.transform(df[numeric_cols])
    categorical_cols = ['weather_condition', 'event_indoor_capability', 'indoor_outdoor_preference']
    for col in categorical_cols:
        df[col] = df[col].fillna('unknown')
    X_cat = encoder.transform(df[categorical_cols])
    X = hstack([tfidf_title_mat, tfidf_interests_mat, X_numeric, X_cat]).toarray()
    y_true = df["interaction_label"].astype(int)
    y_score = model.predict_proba(X)[:, 1]
    return compute_all_metrics(y_true, y_score, k_list=k_list)

def evaluate_svd_on_test(test_df, svd_model, k_list=K_LIST):
    df = test_df.copy()
    df["svd_score"] = df.apply(
        lambda row: svd_model.predict(str(row["user_id"]), str(row["event_id"])).est, axis=1
    )
    y_true = df["interaction_label"].astype(int)
    y_score = df["svd_score"]
    return compute_all_metrics(y_true, y_score, k_list=k_list)



In [4]:
# Load real-world test data
test_users_df = pd.read_csv('/home/nkama/masters_thesis_project/thesis/partially_synthetic/notebooks/test_users_data.csv')
test_events_df = pd.read_csv("/home/nkama/masters_thesis_project/thesis/partially_synthetic/notebooks/test_events_data.csv")
test_interactions_df = pd.read_csv('/home/nkama/masters_thesis_project/thesis/partially_synthetic/notebooks/test_interactions_data.csv')

# Rename columns in test data
test_users_df.rename(columns={
    'lat': 'user_lat',
    'lng': 'user_lon',
    'location': 'user_city',
    'joinedAt': 'signup_date'
}, inplace=True)

test_events_df.rename(columns={
    'category': 'event_type',
    'lat': 'event_lat',
    'lng': 'event_lon',
    'city': 'event_city',
    'weather_description': 'weather_condition',
    'temperature_2m_mean': 'temperature'
}, inplace=True)

test_interactions_df.rename(columns={
    'distance_to_event': 'interaction_distance_to_event'
}, inplace=True)


In [5]:

# Preprocess data
def preprocess_common(interactions_df, users_df, events_df):
    # Create copies of the input dataframes
    interactions_df = interactions_df.copy()
    users_df = users_df.copy()
    events_df = events_df.copy()

    # Drop rows with missing user_id or event_id
    interactions_df = interactions_df.dropna(subset=["user_id", "event_id", "interaction_label"])
    
    # Convert distance_to_event to float
    interactions_df["interaction_distance_to_event"] = interactions_df["interaction_distance_to_event"].fillna(0).astype(float)
    
    # Ensure correct types
    for df in [interactions_df, users_df]:
        df["user_id"] = df["user_id"].astype(str)
    
    for df in [interactions_df, events_df]:
        df["event_id"] = df["event_id"].astype(str)

    interactions_df["interaction_label"] = interactions_df["interaction_label"].astype(int)

    # Convert to string type for TF-IDF fields
    events_df["title"] = events_df["title"].fillna("").astype(str)
    users_df["user_interests"] = users_df["user_interests"].fillna("").astype(str)
    users_df["age"] = users_df["age"].fillna(0).astype(float)

    # Ensure all numeric fields are float
    numeric_cols = ["temperature", "attendance_rate"]
    for col in numeric_cols:
        events_df[col] = pd.to_numeric(events_df[col], errors="coerce").fillna(0).astype(float)

    # Return all three dataframes
    return interactions_df, users_df, events_df


In [6]:
# Preprocess training/validation data
interactions_df, users_df, events_df = preprocess_common(interactions_df, users_df, events_df)
merged_df = interactions_df.merge(events_df, on="event_id").merge(users_df, on="user_id")
merged_df.columns


Index(['interaction_id', 'user_id', 'event_id', 'interaction_type',
       'interaction_time', 'interaction_distance_to_event',
       'interaction_label', 'title', 'event_type', 'event_lat', 'event_lon',
       'event_city', 'start_time', 'duration', 'weather_condition',
       'temperature', 'attendance_rate', 'event_indoor_capability', 'user_lat',
       'user_lon', 'user_city', 'indoor_outdoor_preference', 'age',
       'user_interests', 'signup_date', 'social_connectedness'],
      dtype='object')

In [7]:
# Split into train and validation
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(merged_df, test_size=0.2, random_state=42)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)


In [8]:

# Preprocess test data
test_interactions_df, test_users_df, test_events_df = preprocess_common(test_interactions_df, test_users_df, test_events_df)
test_df = test_interactions_df.merge(test_events_df, on="event_id").merge(test_users_df, on="user_id")
#print(test_df.columns)


In [9]:

# Train models
content_model, content_val_metrics, content_tfidf_title, content_tfidf_interests, content_scaler, content_encoder = train_content_model(train_df, val_df)
svd_model, svd_val_metrics = train_svd(train_df, val_df)
hybrid_model_obj, hybrid_val_metrics, hybrid_tfidf_title, hybrid_tfidf_interests, hybrid_scaler, hybrid_encoder, hybrid_svd_model = train_hybrid_model(train_df, val_df)


In [10]:

# Evaluate on test set
content_test_metrics = evaluate_content_or_hybrid_on_test(
    test_df, content_model, content_tfidf_title, content_tfidf_interests, content_scaler, content_encoder, model_type="content", k_list=K_LIST
)
svd_test_metrics = evaluate_svd_on_test(test_df, svd_model, k_list=K_LIST)
hybrid_test_metrics = evaluate_content_or_hybrid_on_test(
    test_df, hybrid_model_obj, hybrid_tfidf_title, hybrid_tfidf_interests, hybrid_scaler, hybrid_encoder, model_type="hybrid", svd_model=hybrid_svd_model, k_list=K_LIST
)


In [11]:

# --- Save results in a DataFrame (excluding AUC and MAP) ---
def filter_metrics(metrics_dict):
    return {k: v for k, v in metrics_dict.items() if not (k.startswith("AUC") or k.startswith("MAP"))}

results_df = pd.DataFrame({
    "Content-Based": filter_metrics(content_test_metrics),
    "SVD": filter_metrics(svd_test_metrics),
    "Hybrid": filter_metrics(hybrid_test_metrics),
}).T

results_df.to_csv("test_metrics_k_results.csv")
print(results_df)


               Precision@1  Recall@1  NDCG@1  MRR@1  Precision@5  Recall@5  \
Content-Based          1.0  0.000038     1.0    1.0          1.0  0.000188   
SVD                    1.0  0.000038     1.0    1.0          1.0  0.000188   
Hybrid                 1.0  0.000038     1.0    1.0          0.8  0.000150   

                 NDCG@5  MRR@5  Precision@10  Recall@10   NDCG@10  MRR@10  \
Content-Based  1.000000    1.0           1.0   0.000375  1.000000     1.0   
SVD            1.000000    1.0           1.0   0.000375  1.000000     1.0   
Hybrid         0.853932    1.0           0.9   0.000338  0.905212     1.0   

               Precision@50  Recall@50   NDCG@50  MRR@50  Precision@100  \
Content-Based          0.96   0.001801  0.969856     1.0           0.93   
SVD                    0.96   0.001801  0.966914     1.0           0.86   
Hybrid                 0.94   0.001763  0.930284     1.0           0.96   

               Recall@100  NDCG@100  MRR@100  
Content-Based    0.003489  0.9

In [ ]:
results_df

,Precision@1,Recall@1,NDCG@1,MRR@1,Precision@5,Recall@5,NDCG@5,MRR@5,Precision@10,Recall@10,NDCG@10,MRR@10,Precision@50,Recall@50,NDCG@50,MRR@50,Precision@100,Recall@100,NDCG@100,MRR@100
Content-Based,1.0,0.000038,1.0,1.0,1.0,0.000188,1.000000,1.0,1.0,0.000375,1.000000,1.0,0.96,0.001801,0.969856,1.0,0.93,0.003489,0.943068,1.0
SVD,1.0,0.000038,1.0,1.0,1.0,0.000188,1.000000,1.0,1.0,0.000375,1.000000,1.0,0.96,0.001801,0.966914,1.0,0.86,0.003226,0.886381,1.0
Hybrid,1.0,0.000038,1.0,1.0,0.8,0.000150,0.853932,1.0,0.9,0.000338,0.905212,1.0,0.94,0.001763,0.930284,1.0,0.96,0.003601,0.949067,1.0


: 

In [ ]:

import hopsworks
# Connect to Hopsworks Feature Store
project = hopsworks.login()
fs = project.get_feature_store()
# Load from Hopsworks feature groups
users_fg = fs.get_feature_group(name="users", version=1)
events_fg = fs.get_feature_group(name="events", version=1)
interactions_fg = fs.get_feature_group(name="interactions", version=1)

#read data from hopsworks
users_df = users_fg.read()
events_df = events_fg.read()
interactions_df =interactions_fg.read()
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from surprise import Dataset, Reader, SVD
from scipy.sparse import hstack
from catboost import CatBoostClassifier

# --- Metrics ---
K_LIST = [1, 5, 10, 50, 100]

def compute_ranking_metrics(y_true, y_score, k=10):
    sorted_indices = np.argsort(y_score)[::-1]
    top_k = np.array(y_true)[sorted_indices][:k]
    precision = np.mean(top_k)
    recall = np.sum(top_k) / np.sum(y_true) if np.sum(y_true) > 0 else 0
    dcg = np.sum(top_k / np.log2(np.arange(2, len(top_k) + 2)))
    ideal_k = min(int(np.sum(y_true)), k)
    idcg = np.sum([1 / np.log2(i + 2) for i in range(ideal_k)])
    ndcg = dcg / idcg if idcg > 0 else 0
    return precision, recall, ndcg

def mean_reciprocal_rank(y_true, y_score, k=100):
    sorted_indices = np.argsort(y_score)[::-1][:k]
    top_k = np.array(y_true)[sorted_indices]
    ranks = np.where(top_k == 1)[0]
    if len(ranks) == 0:
        return 0.0
    return 1.0 / (ranks[0] + 1)

def compute_all_metrics(y_true, y_score, k_list=K_LIST):
    metrics = {}
    for k in k_list:
        p, r, n = compute_ranking_metrics(y_true, y_score, k)
        metrics[f"Precision@{k}"] = p
        metrics[f"Recall@{k}"] = r
        metrics[f"NDCG@{k}"] = n
        metrics[f"MRR@{k}"] = mean_reciprocal_rank(y_true, y_score, k)
    return metrics

# --- Model Training ---
def train_content_model(train_df, val_df):
    train_clean = train_df.copy()
    val_clean = val_df.copy()
    for df in [train_clean, val_clean]:
        df["title"] = df["title"].fillna("")
        df["user_interests"] = df["user_interests"].fillna("")
    tfidf_title = TfidfVectorizer(max_features=100)
    tfidf_interests = TfidfVectorizer(max_features=100)
    tfidf_title_mat_train = tfidf_title.fit_transform(train_clean["title"])
    tfidf_interests_mat_train = tfidf_interests.fit_transform(train_clean["user_interests"])
    tfidf_title_mat_val = tfidf_title.transform(val_clean["title"])
    tfidf_interests_mat_val = tfidf_interests.transform(val_clean["user_interests"])
    numeric_cols = ["interaction_distance_to_event", "temperature", "age", "attendance_rate"]
    for df in [train_clean, val_clean]:
        df[numeric_cols] = df[numeric_cols].fillna(0)
    scaler = StandardScaler().fit(train_clean[numeric_cols])
    X_numeric_train = scaler.transform(train_clean[numeric_cols])
    X_numeric_val = scaler.transform(val_clean[numeric_cols])
    categorical_cols = ['weather_condition', 'event_indoor_capability', 'indoor_outdoor_preference']
    for df in [train_clean, val_clean]:
        for col in categorical_cols:
            df[col] = df[col].fillna('unknown')
    encoder = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
    X_cat_train = encoder.fit_transform(train_clean[categorical_cols])
    X_cat_val = encoder.transform(val_clean[categorical_cols])
    X_train = hstack([tfidf_title_mat_train, tfidf_interests_mat_train, X_numeric_train, X_cat_train]).toarray()
    X_val = hstack([tfidf_title_mat_val, tfidf_interests_mat_val, X_numeric_val, X_cat_val]).toarray()
    y_train = train_clean["interaction_label"].astype(int)
    y_val = val_clean["interaction_label"].astype(int)
    model = CatBoostClassifier(iterations=200, depth=6, learning_rate=0.1, loss_function='Logloss', verbose=False)
    model.fit(X_train, y_train)
    val_scores = model.predict_proba(X_val)[:, 1]
    return model, compute_all_metrics(y_val, val_scores), tfidf_title, tfidf_interests, scaler, encoder

def train_svd(train_df, val_df):
    reader = Reader(rating_scale=(0, 1))
    train_svd = train_df[["user_id", "event_id", "interaction_label"]].copy()
    train_svd["user_id"] = train_svd["user_id"].astype(str)
    train_svd["event_id"] = train_svd["event_id"].astype(str)
    train_svd["interaction_label"] = train_svd["interaction_label"].astype(float)
    data = Dataset.load_from_df(train_svd, reader)
    trainset = data.build_full_trainset()
    svd = SVD(n_epochs=50).fit(trainset)
    val_copy = val_df.copy()
    val_copy["svd_score"] = val_copy.apply(
        lambda row: svd.predict(str(row["user_id"]), str(row["event_id"])).est, axis=1
    )
    y_val = val_copy["interaction_label"].astype(int)
    val_scores = val_copy["svd_score"]
    return svd, compute_all_metrics(y_val, val_scores)

def train_hybrid_model(train_df, val_df):
    reader = Reader(rating_scale=(0, 1))
    train_svd_data = Dataset.load_from_df(train_df[["user_id", "event_id", "interaction_label"]].astype(str), reader)
    trainset = train_svd_data.build_full_trainset()
    svd_model = SVD(n_epochs=20).fit(trainset)
    train_df = train_df.copy()
    val_df = val_df.copy()
    train_df["svd_score"] = train_df.apply(
        lambda row: svd_model.predict(str(row["user_id"]), str(row["event_id"])).est, axis=1
    )
    val_df["svd_score"] = val_df.apply(
        lambda row: svd_model.predict(str(row["user_id"]), str(row["event_id"])).est, axis=1
    )
    tfidf_title = TfidfVectorizer(max_features=50)
    tfidf_interests = TfidfVectorizer(max_features=50)
    tfidf_title.fit(train_df["title"].fillna(""))
    tfidf_interests.fit(train_df["user_interests"].fillna(""))
    X_train_text = hstack([
        tfidf_title.transform(train_df["title"].fillna("")),
        tfidf_interests.transform(train_df["user_interests"].fillna(""))
    ])
    X_val_text = hstack([
        tfidf_title.transform(val_df["title"].fillna("")),
        tfidf_interests.transform(val_df["user_interests"].fillna(""))
    ])
    numeric_cols = ["interaction_distance_to_event", "temperature", "age", "attendance_rate", "svd_score"]
    for df in [train_df, val_df]:
        df[numeric_cols] = df[numeric_cols].fillna(0)
    scaler = StandardScaler().fit(train_df[numeric_cols])
    X_train_numeric = scaler.transform(train_df[numeric_cols])
    X_val_numeric = scaler.transform(val_df[numeric_cols])
    categorical_cols = ['weather_condition', 'event_indoor_capability', 'indoor_outdoor_preference']
    for df in [train_df, val_df]:
        for col in categorical_cols:
            df[col] = df[col].fillna('unknown')
    encoder = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
    X_cat_train = encoder.fit_transform(train_df[categorical_cols])
    X_cat_val = encoder.transform(val_df[categorical_cols])
    X_train = hstack([X_train_text, X_train_numeric, X_cat_train]).toarray()
    X_val = hstack([X_val_text, X_val_numeric, X_cat_val]).toarray()
    y_train = train_df["interaction_label"].astype(int)
    y_val = val_df["interaction_label"].astype(int)
    model = CatBoostClassifier(iterations=200, depth=6, learning_rate=0.1, loss_function='Logloss', verbose=False)
    model.fit(X_train, y_train)
    val_scores = model.predict_proba(X_val)[:, 1]
    return model, compute_all_metrics(y_val, val_scores), tfidf_title, tfidf_interests, scaler, encoder, svd_model

# --- Test Set Evaluation ---
def evaluate_content_or_hybrid_on_test(test_df, model, tfidf_title, tfidf_interests, scaler, encoder, model_type="content", svd_model=None, k_list=K_LIST):
    df = test_df.copy()
    df["title"] = df["title"].fillna("")
    df["user_interests"] = df["user_interests"].fillna("")
    if model_type == "hybrid":
        df["svd_score"] = df.apply(
            lambda row: svd_model.predict(str(row["user_id"]), str(row["event_id"])).est, axis=1
        )
        numeric_cols = ["interaction_distance_to_event", "temperature", "age", "attendance_rate", "svd_score"]
    else:
        numeric_cols = ["interaction_distance_to_event", "temperature", "age", "attendance_rate"]
    df[numeric_cols] = df[numeric_cols].fillna(0)
    tfidf_title_mat = tfidf_title.transform(df["title"])
    tfidf_interests_mat = tfidf_interests.transform(df["user_interests"])
    X_numeric = scaler.transform(df[numeric_cols])
    categorical_cols = ['weather_condition', 'event_indoor_capability', 'indoor_outdoor_preference']
    for col in categorical_cols:
        df[col] = df[col].fillna('unknown')
    X_cat = encoder.transform(df[categorical_cols])
    X = hstack([tfidf_title_mat, tfidf_interests_mat, X_numeric, X_cat]).toarray()
    y_true = df["interaction_label"].astype(int)
    y_score = model.predict_proba(X)[:, 1]
    return compute_all_metrics(y_true, y_score, k_list=k_list)

def evaluate_svd_on_test(test_df, svd_model, k_list=K_LIST):
    df = test_df.copy()
    df["svd_score"] = df.apply(
        lambda row: svd_model.predict(str(row["user_id"]), str(row["event_id"])).est, axis=1
    )
    y_true = df["interaction_label"].astype(int)
    y_score = df["svd_score"]
    return compute_all_metrics(y_true, y_score, k_list=k_list)


# Load real-world test data
test_users_df = pd.read_csv('/home/nkama/masters_thesis_project/thesis/partially_synthetic/notebooks/test_users_data.csv')
test_events_df = pd.read_csv("/home/nkama/masters_thesis_project/thesis/partially_synthetic/notebooks/test_events_data.csv")
test_interactions_df = pd.read_csv('/home/nkama/masters_thesis_project/thesis/partially_synthetic/notebooks/test_interactions_data.csv')

# Rename columns in test data
test_users_df.rename(columns={
    'lat': 'user_lat',
    'lng': 'user_lon',
    'location': 'user_city',
    'joinedAt': 'signup_date'
}, inplace=True)

test_events_df.rename(columns={
    'category': 'event_type',
    'lat': 'event_lat',
    'lng': 'event_lon',
    'city': 'event_city',
    'weather_description': 'weather_condition',
    'temperature_2m_mean': 'temperature'
}, inplace=True)

test_interactions_df.rename(columns={
    'distance_to_event': 'interaction_distance_to_event'
}, inplace=True)


# Preprocess data
def preprocess_common(interactions_df, users_df, events_df):
    # Create copies of the input dataframes
    interactions_df = interactions_df.copy()
    users_df = users_df.copy()
    events_df = events_df.copy()

    # Drop rows with missing user_id or event_id
    interactions_df = interactions_df.dropna(subset=["user_id", "event_id", "interaction_label"])
    
    # Convert distance_to_event to float
    interactions_df["interaction_distance_to_event"] = interactions_df["interaction_distance_to_event"].fillna(0).astype(float)
    
    # Ensure correct types
    for df in [interactions_df, users_df]:
        df["user_id"] = df["user_id"].astype(str)
    
    for df in [interactions_df, events_df]:
        df["event_id"] = df["event_id"].astype(str)

    interactions_df["interaction_label"] = interactions_df["interaction_label"].astype(int)

    # Convert to string type for TF-IDF fields
    events_df["title"] = events_df["title"].fillna("").astype(str)
    users_df["user_interests"] = users_df["user_interests"].fillna("").astype(str)
    users_df["age"] = users_df["age"].fillna(0).astype(float)

    # Ensure all numeric fields are float
    numeric_cols = ["temperature", "attendance_rate"]
    for col in numeric_cols:
        events_df[col] = pd.to_numeric(events_df[col], errors="coerce").fillna(0).astype(float)

    # Return all three dataframes
    return interactions_df, users_df, events_df

# Preprocess training/validation data
interactions_df, users_df, events_df = preprocess_common(interactions_df, users_df, events_df)
merged_df = interactions_df.merge(events_df, on="event_id").merge(users_df, on="user_id")
merged_df.columns

# Split into train and validation
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(merged_df, test_size=0.2, random_state=42)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)


# Preprocess test data
test_interactions_df, test_users_df, test_events_df = preprocess_common(test_interactions_df, test_users_df, test_events_df)
test_df = test_interactions_df.merge(test_events_df, on="event_id").merge(test_users_df, on="user_id")
#print(test_df.columns)


# Train models
content_model, content_val_metrics, content_tfidf_title, content_tfidf_interests, content_scaler, content_encoder = train_content_model(train_df, val_df)
svd_model, svd_val_metrics = train_svd(train_df, val_df)
hybrid_model_obj, hybrid_val_metrics, hybrid_tfidf_title, hybrid_tfidf_interests, hybrid_scaler, hybrid_encoder, hybrid_svd_model = train_hybrid_model(train_df, val_df)


# Evaluate on test set
content_test_metrics = evaluate_content_or_hybrid_on_test(
    test_df, content_model, content_tfidf_title, content_tfidf_interests, content_scaler, content_encoder, model_type="content", k_list=K_LIST
)
svd_test_metrics = evaluate_svd_on_test(test_df, svd_model, k_list=K_LIST)
hybrid_test_metrics = evaluate_content_or_hybrid_on_test(
    test_df, hybrid_model_obj, hybrid_tfidf_title, hybrid_tfidf_interests, hybrid_scaler, hybrid_encoder, model_type="hybrid", svd_model=hybrid_svd_model, k_list=K_LIST
)


# --- Save results in a DataFrame (excluding AUC and MAP) ---
def filter_metrics(metrics_dict):
    return {k: v for k, v in metrics_dict.items() if not (k.startswith("AUC") or k.startswith("MAP"))}

results_df = pd.DataFrame({
    "Content-Based": filter_metrics(content_test_metrics),
    "SVD": filter_metrics(svd_test_metrics),
    "Hybrid": filter_metrics(hybrid_test_metrics),
}).T

results_df.to_csv("test_metrics_k_results.csv")
print(results_df)

results_df